In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
import tflearn
import poisson_extensions
poisson_extensions.register_poisson_gradient(tf, ops)

In [2]:
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [3]:
# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

In [4]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [5]:
# Convolutional network building, this dropout model achieves .8 accuracy, same for poisson used in final 2 layers
inputs = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
conv1 = conv_2d(inputs, 32, 3, activation='leaky_relu')
conv1 = conv_2d(inputs, 32, 3, activation='leaky_relu')
pool1 = max_pool_2d(conv1, 2)
spike1 = dropout(pool1, 1.)
# spike1 = tf.random_poisson(pool1, [])
after1 = tf.identity(spike1)
conv2 = conv_2d(after1, 64, 3, activation='leaky_relu')
conv2 = conv_2d(conv2, 64, 3, activation='leaky_relu')
pool2 = max_pool_2d(conv2, 2)
spike2 = dropout(pool2, 1.)
# spike2 = tf.random_poisson(pool2, [])
# after2 = tf.identity(spike2)
conv3 = conv_2d(spike2, 128, 3, activation='leaky_relu')
conv3a = conv_2d(conv3, 128, 3, activation='leaky_relu')
pool3 = max_pool_2d(conv3, 2, name='pool3')
# spike3 = dropout(pool3, 0.6)
spike3 = tf.random_poisson(pool3, [], name='spike3')
# after3 = tf.identity(spike3, name='after3')
fc1 = fully_connected(spike3, 512, activation='relu', name='fc1')
# spike4 = dropout(fc1, 0.5)
spike4 = tf.random_poisson(fc1, [], name='spike4')
# after4 = tf.identity(spike4, name='after4')
fc2 = fully_connected(spike4, 10, activation='softmax')
net = regression(fc2, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

In [6]:
# tflearn.add_weights_regularizer(conv2, 'L2', weight_decay=0.001)
# tflearn.add_weights_regularizer(conv1, 'L2', weight_decay=0.001)
# tflearn.add_weights_regularizer(conv3a, 'L2', weight_decay=0.001)
tflearn.add_weights_regularizer(fc1, 'L2', weight_decay=0.0001)

<tf.Tensor 'L2-Loss:0' shape=() dtype=float32>

In [7]:
tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, fc1.W)
tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, fc1.b)

In [8]:
model = tflearn.DNN(net, tensorboard_verbose=0)

In [9]:
model.fit(X, Y, n_epoch=26, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn_poisson_regularize_test')

Training Step: 458  | total loss: 1.42414 | time: 18.795s
| Adam | epoch: 001 | loss: 1.42414 - acc: 0.4944 -- iter: 43968/50000


KeyboardInterrupt: 

In [ ]:
model.session.close()

In [10]:
import tensorflow.contrib.graph_editor as ge

In [11]:
rand1 = ge.make_view('spike3',graph=tf.get_default_graph())
rand2 = ge.make_view('spike4',graph=tf.get_default_graph())

In [ ]:
rand2.outputs[:]

In [12]:
ge.bypass(rand1)

(<tensorflow.contrib.graph_editor.subgraph.SubGraphView at 0x7f25a7772438>,
 [<tf.Tensor 'pool3/geph__MaxPool_0:0' shape=<unknown> dtype=float32>])

In [13]:
ge.bypass(rand2)

(<tensorflow.contrib.graph_editor.subgraph.SubGraphView at 0x7f25cc654c50>,
 [<tf.Tensor 'fc1/geph__Relu_0:0' shape=<unknown> dtype=float32>])

In [23]:
model.evaluate(X=X_test,Y=Y_test)

[0.48459999999999998]

In [24]:
model2 = tflearn.DNN(net, tensorboard_verbose=0)

In [27]:
model2.evaluate(X=X_test, Y=Y_test)

[0.48420000000000002]